This notebook demonstrates 1d sharded checkpoint offload.

Profile (v6e-8): http://shortn/_AVBaAqnFNd

In [1]:
%env PJRT_DEVICE=TPU
%env XLA_IR_DEBUG=1
%env XLA_HLO_DEBUG=1
%env TPU_LIBRARY_PATH=/workspaces/torch/_libtpu.so

# MaxText flags except that we disable optimization barrier removal: crashes in native code
%env LIBTPU_INIT_ARGS=--xla_enable_async_all_gather=true --xla_tpu_enable_async_collective_fusion=true --xla_tpu_enable_async_collective_fusion_fuse_all_gather=true --xla_tpu_enable_async_collective_fusion_multiple_steps=true --xla_tpu_decompose_all_gather_einsum=true --xla_tpu_decompose_einsum_reduce_scatter=true --xla_tpu_scoped_vmem_limit_kib=98304 --xla_tpu_spmd_rng_bit_generator_unsafe=true --xla_tpu_overlap_compute_collective_tc=true --xla_tpu_use_enhanced_launch_barrier=true --xla_tpu_enable_all_experimental_scheduler_features=true --xla_tpu_enable_scheduler_memory_pressure_tracking=true --xla_tpu_host_transfer_overlap_limit=2 --xla_tpu_aggressive_opt_barrier_removal=DISABLED --xla_lhs_prioritize_async_depth_over_stall=ENABLED --xla_tpu_enable_ag_backward_pipelining=true --xla_should_allow_loop_variant_parameter_in_chain=ENABLED --xla_should_add_loop_invariant_op_in_chain=ENABLED --xla_max_concurrent_host_send_recv=100 --xla_tpu_scheduler_percent_shared_memory_limit=100 --xla_latency_hiding_scheduler_rerun=2

# Still crashes in native code
# %env LIBTPU_INIT_ARGS=--xla_enable_async_all_gather=true --xla_tpu_enable_async_collective_fusion=true --xla_tpu_enable_async_collective_fusion_fuse_all_gather=true --xla_tpu_enable_async_collective_fusion_multiple_steps=true --xla_tpu_decompose_all_gather_einsum=true --xla_tpu_decompose_einsum_reduce_scatter=true --xla_tpu_scoped_vmem_limit_kib=98304 --xla_tpu_spmd_rng_bit_generator_unsafe=true --xla_tpu_overlap_compute_collective_tc=true --xla_tpu_use_enhanced_launch_barrier=true --xla_tpu_enable_scheduler_memory_pressure_tracking=true --xla_tpu_host_transfer_overlap_limit=2 --xla_tpu_aggressive_opt_barrier_removal=DISABLED --xla_lhs_prioritize_async_depth_over_stall=ENABLED --xla_tpu_enable_ag_backward_pipelining=true --xla_should_allow_loop_variant_parameter_in_chain=ENABLED --xla_should_add_loop_invariant_op_in_chain=ENABLED --xla_max_concurrent_host_send_recv=100 --xla_tpu_scheduler_percent_shared_memory_limit=100

# Removed some more flags, and 1% scheduler shared memory limit: OK
# %env LIBTPU_INIT_ARGS=--xla_tpu_overlap_compute_collective_tc=true --xla_tpu_use_enhanced_launch_barrier=true --xla_tpu_enable_scheduler_memory_pressure_tracking=true --xla_tpu_host_transfer_overlap_limit=2 --xla_tpu_aggressive_opt_barrier_removal=DISABLED --xla_lhs_prioritize_async_depth_over_stall=ENABLED --xla_tpu_enable_ag_backward_pipelining=true --xla_should_allow_loop_variant_parameter_in_chain=ENABLED --xla_should_add_loop_invariant_op_in_chain=ENABLED --xla_max_concurrent_host_send_recv=100 --xla_tpu_scheduler_percent_shared_memory_limit=1

env: PJRT_DEVICE=TPU
env: XLA_IR_DEBUG=1
env: XLA_HLO_DEBUG=1
env: TPU_LIBRARY_PATH=/workspaces/torch/_libtpu.so
env: LIBTPU_INIT_ARGS=--xla_enable_async_all_gather=true --xla_tpu_enable_async_collective_fusion=true --xla_tpu_enable_async_collective_fusion_fuse_all_gather=true --xla_tpu_enable_async_collective_fusion_multiple_steps=true --xla_tpu_decompose_all_gather_einsum=true --xla_tpu_decompose_einsum_reduce_scatter=true --xla_tpu_scoped_vmem_limit_kib=98304 --xla_tpu_spmd_rng_bit_generator_unsafe=true --xla_tpu_overlap_compute_collective_tc=true --xla_tpu_use_enhanced_launch_barrier=true --xla_tpu_enable_all_experimental_scheduler_features=true --xla_tpu_enable_scheduler_memory_pressure_tracking=true --xla_tpu_host_transfer_overlap_limit=2 --xla_tpu_aggressive_opt_barrier_removal=DISABLED --xla_lhs_prioritize_async_depth_over_stall=ENABLED --xla_tpu_enable_ag_backward_pipelining=true --xla_should_allow_loop_variant_parameter_in_chain=ENABLED --xla_should_add_loop_invariant_op_in_c

In [2]:
import torch_xla
import torch
from torch_xla import runtime as xr

In [3]:
from torch.autograd.graph import saved_tensors_hooks
from torch_xla.experimental.stablehlo_custom_call import (
  place_to_host, place_to_device
)

class OffloadingModule(torch.nn.Module):
  def __init__(self, m):
    super().__init__()
    self.m = m

  def forward(self, *args, **kwargs):
    with saved_tensors_hooks(place_to_host, place_to_device):
      return self.m(*args, **kwargs)

In [4]:
import decoder_only_model
from trainer import TrainDecoderOnlyBase
import functools

## Test 1D FSDP sharding

In [5]:
import torch
import numpy as np
import torch_xla.distributed.spmd as xs
import torch_xla.utils.utils as xu
import torch_xla.distributed.parallel_loader as pl
from torch_xla.experimental.spmd_fully_sharded_data_parallel import SpmdFullyShardedDataParallel as FSDPv2
from torch_xla import runtime as xr
from torch_xla.distributed.fsdp.wrap import transformer_auto_wrap_policy

# checkout our doc at https://github.com/pytorch/xla/blob/master/docs/fsdpv2.md
class TrainDecoderOnlyFSDPv2(TrainDecoderOnlyBase):

  def __init__(self):
    super().__init__(decoder_only_model.DecoderOnlyConfig(
       hidden_size=4096,
       num_hidden_layers=32,
       num_attention_heads=16,
       num_key_value_heads=8,
       intermediate_size=8192,
       vocab_size=16384,
    ))
    # Define the mesh following common SPMD practice
    num_devices = xr.global_runtime_device_count()
    mesh_shape = (num_devices, 1)
    device_ids = np.array(range(num_devices))
    # To be noted, the mesh must have an axis named 'fsdp', which the weights and activations will be sharded on.
    mesh = xs.Mesh(device_ids, mesh_shape, ('fsdp', 'model'))
    xs.set_global_mesh(mesh)

    # Shard the input(data parallel).
    # Scale the batch size with num_devices since there will be only one
    # process that handles all runtime devices.
    self.batch_size *= num_devices
    train_loader = xu.SampleGenerator(
        data=(torch.randint(
            0,
            self.config.vocab_size, (self.batch_size, self.seq_len),
            dtype=torch.int64,
            device='cpu'),
              torch.randint(
                  0,
                  self.config.vocab_size, (self.batch_size, self.seq_len),
                  dtype=torch.int64,
                  device='cpu')),
        sample_count=self.train_dataset_len // self.batch_size)
    self.train_device_loader = pl.MpDeviceLoader(
        train_loader,
        self.device,
        # Shard the input's batch dimension along the `fsdp` axis, no sharding along other dimensions
        input_sharding=xs.ShardingSpec(mesh, ('fsdp', None)))  # type:ignore
    
    model: decoder_only_model.DecoderOnlyModel = self.model  # type:ignore
    self.model = model

    # Apply checkpoint to each DecoderLayer layer.
    from torch_xla.distributed.fsdp import checkpoint_module
    for i, block in enumerate(self.model.layers):
        self.model.layers[i] = checkpoint_module(block)
        
    # Apply offloading to each DecoderLayer layer.
    from torch_xla.distributed.fsdp import checkpoint_module
    for i, block in enumerate(self.model.layers):
        self.model.layers[i] = OffloadingModule(block)

    # Apply FSDP sharding on each DecoderLayer layer.
    auto_wrap_policy = functools.partial(
        transformer_auto_wrap_policy,
        transformer_layer_cls={
            OffloadingModule
        },
    )

    # FSDPv2 will use the global mesh set above
    self.model: torch.nn.Module = self.model
    self.model = FSDPv2(
        self.model, auto_wrap_policy=auto_wrap_policy)
    self.optimizer = torch.optim.SGD(self.model.parameters(), lr=0.00001)


In [ ]:
xr.use_spmd()
base = TrainDecoderOnlyFSDPv2()

: 

In [ ]:
print("Compiling model")
base.num_steps = 3
base.start_training()
torch_xla.sync(wait=True)

print("Profiling model")
import torch_xla.debug.profiler as xp
server = xp.start_server(9012)
xp.trace_detached(
    service_addr="localhost:9012", logdir=f"profile", duration_ms=15000)
base.num_steps = 5
base.start_training()
torch_xla.sync(wait=True)
del server

Compiling model
Epoch 1 train begin  5:41AM UTC on Nov 10, 2024


/workspaces/torch/pytorch/xla/torch_xla/utils/checkpoint.py:183: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  torch.cuda.amp.autocast(**ctx.gpu_autocast_kwargs), \
/workspaces/torch/pytorch/xla/torch_xla/utils/checkpoint.py:184: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):
https://symbolize.corp.google.com/r/?trace=79fe4f0fb0ad,7a0fb32a8dcf,79fe4f0fa2bc,79fe4f1020c3,79fe5408cc11,79fe5408c678,79fe4f0a6ee1,79fe4f09ceb0,79fe517c41b2,79fe56af4d12,79fe56afaed5,79fe56b03a44,79fe56c9f602,7a0fb3255ea6&map= 
*** SIGSEGV (@0x18), see go/stacktraces#s15 received by PID 2076220 (TID 2077780) on cpu 5; stack trace: ***
PC: @     0x79fe4f0fb0ad  (unknown)  xla::jellyfish::(anonymous namespace)::HasActivationSemantics()
    @     0x79fe56dc80e1       1888  FailureSignalHandler()
    @     0x7a0f

In [ ]:
from visualize import visualize_tensor_sharding
_ = visualize_tensor_sharding(base.model.embed_tokens.weight, use_color=False)

4096   
     ┌───────┐
     │       │
     │ TPU 0 │
     │       │
     ├───────┤
     │       │
     │ TPU 1 │
     │       │
     ├───────┤
     │       │
     │ TPU 2 │
     │       │
     ├───────┤
     │       │
     │ TPU 3 │
     │       │
16384├───────┤
     │       │
     │ TPU 4 │
     │       │
     ├───────┤
     │       │
     │ TPU 5 │
     │       │
     ├───────┤
     │       │
     │ TPU 6 │
     │       │
     ├───────┤
     │       │
     │ TPU 7 │
     │       │
     └───────┘

In [ ]:
_ = visualize_tensor_sharding(base.model.layers[0].m.self_attn.q_proj.weight, use_color=False)

4096   
    ┌───────┐
    │       │
    │ TPU 0 │
    │       │
    ├───────┤
    │       │
    │ TPU 1 │
    │       │
    ├───────┤
    │       │
    │ TPU 2 │
    │       │
    ├───────┤
    │       │
    │ TPU 3 │
    │       │
4096├───────┤
    │       │
    │ TPU 4 │
    │       │
    ├───────┤
    │       │
    │ TPU 5 │
    │       │
    ├───────┤
    │       │
    │ TPU 6 │
    │       │
    ├───────┤
    │       │
    │ TPU 7 │
    │       │
    └───────┘

In [ ]:
_ = visualize_tensor_sharding(base.model.layers[0].m.self_attn.o_proj.weight, use_color=False)

4096   
    ┌───────┐
    │       │
    │ TPU 0 │
    │       │
    ├───────┤
    │       │
    │ TPU 1 │
    │       │
    ├───────┤
    │       │
    │ TPU 2 │
    │       │
    ├───────┤
    │       │
    │ TPU 3 │
    │       │
4096├───────┤
    │       │
    │ TPU 4 │
    │       │
    ├───────┤
    │       │
    │ TPU 5 │
    │       │
    ├───────┤
    │       │
    │ TPU 6 │
    │       │
    ├───────┤
    │       │
    │ TPU 7 │
    │       │
    └───────┘